In [ ]:
import tkinter as tk
import numpy as np
import random
import time
from queue import Queue, PriorityQueue

# Initialize global variables
GRID_SIZE = 20
CANVAS_SIZE = 600
BUTTON_HEIGHT = 50

# Maze sizes for small, medium, and large
SMALL_MAZE = 10
MEDIUM_MAZE = 20
LARGE_MAZE = 30

class MazeSolverApp:
    def __init__(self, root):
        self.root = root
        self.root.title("AI Maze Solver")

        # Create canvas
        self.canvas = tk.Canvas(self.root, width=CANVAS_SIZE, height=CANVAS_SIZE, bg='white')
        self.canvas.grid(row=0, column=0, columnspan=6)

        # Add control buttons
        self.random_button = tk.Button(self.root, text="Random Maze", command=self.generate_random_maze)
        self.random_button.grid(row=1, column=0)

        self.dfs_button = tk.Button(self.root, text="Solve with DFS", command=self.solve_with_dfs)
        self.dfs_button.grid(row=1, column=1)

        self.bfs_button = tk.Button(self.root, text="Solve with BFS", command=self.solve_with_bfs)
        self.bfs_button.grid(row=1, column=2)

        self.astar_button = tk.Button(self.root, text="Solve with A*", command=self.solve_with_astar)
        self.astar_button.grid(row=1, column=3)

        self.iddfs_button = tk.Button(self.root, text="Solve with IDDFS", command=self.solve_with_iddfs)
        self.iddfs_button.grid(row=1, column=4)

        self.size_label = tk.Label(self.root, text="Maze Size:")
        self.size_label.grid(row=1, column=5)

        self.size_var = tk.StringVar(value="Small")
        self.size_menu = tk.OptionMenu(self.root, self.size_var, "Small", "Medium", "Large", command=self.change_size)
        self.size_menu.grid(row=1, column=6)

        # Initial maze size and maze generation
        self.maze_size = SMALL_MAZE
        self.maze = self.generate_maze(self.maze_size)
        self.solution_path = None
        self.draw_maze()

    def change_size(self, size_text):
        """Changes maze size based on user selection"""
        if size_text == "Small":
            self.maze_size = SMALL_MAZE
        elif size_text == "Medium":
            self.maze_size = MEDIUM_MAZE
        else:
            self.maze_size = LARGE_MAZE

        self.generate_random_maze()

    def generate_random_maze(self):
        """Generates a random maze and redraws it"""
        self.maze = self.generate_maze(self.maze_size)
        self.solution_path = None
        self.draw_maze()

    def generate_maze(self, size):
        """Generates a solvable maze by first creating a guaranteed path and then adding random walls."""
        maze = np.ones((size, size), dtype=int)  # Start with a grid full of walls
        start = (0, 0)
        maze[0][0] = 0  # Start point
        maze[size - 1][size - 1] = 0  # End point

        # Step 1: Create a guaranteed path using DFS
        path = []
        visited = set()

        def dfs_generate(x, y):
            path.append((x, y))
            visited.add((x, y))
            maze[x][y] = 0

            if (x, y) == (size - 1, size - 1):
                return True  # Goal reached

            directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
            random.shuffle(directions)

            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 <= nx < size and 0 <= ny < size and (nx, ny) not in visited:
                    if dfs_generate(nx, ny):
                        return True

            return False

        dfs_generate(0, 0)

        # Step 2: Add random walls to the rest of the maze, except for the path
        for i in range(size):
            for j in range(size):
                if (i, j) not in path and random.random() > 0.3:  # 30% chance to be a path, 70% chance to be a wall
                    maze[i][j] = 1  # Wall

        return maze

    def draw_maze(self):
        """Draws the maze and solution path on the canvas"""
        self.canvas.delete("all")
        size = len(self.maze)
        cell_size = CANVAS_SIZE // size
        for i in range(size):
            for j in range(size):
                if (i, j) == (0, 0):
                    color = 'green'  # Start point
                elif (i, j) == (size - 1, size - 1):
                    color = 'red'  # End point
                elif self.maze[i][j] == 1:
                    color = 'black'  # Wall
                else:
                    color = 'white'  # Path
                self.canvas.create_rectangle(j * cell_size, i * cell_size, (j + 1) * cell_size, (i + 1) * cell_size, fill=color)

        if self.solution_path:
            # Slow down and show the blue path search
            for (i, j) in self.solution_path:
                self.canvas.create_rectangle(j * cell_size, i * cell_size, (j + 1) * cell_size, (i + 1) * cell_size, fill='blue')
                self.root.update()
                time.sleep(0.05)  # Slow down to show path search

            # Animate the path turning green
            time.sleep(1)  # Pause before turning the path green
            for (i, j) in self.solution_path:
                self.canvas.create_rectangle(j * cell_size, i * cell_size, (j + 1) * cell_size, (i + 1) * cell_size, fill='green')
                self.root.update()
                time.sleep(0.05)

    def solve_with_dfs(self):
        start_time = time.time()
        self.solution_path = dfs(self.maze)
        elapsed_time = time.time() - start_time
        self.show_solution(elapsed_time, "DFS")
        time.sleep(1)  # Pause for 1 second before continuing

    def solve_with_bfs(self):
        start_time = time.time()
        self.solution_path = bfs(self.maze)
        elapsed_time = time.time() - start_time
        self.show_solution(elapsed_time, "BFS")
        time.sleep(1)  # Pause for 1 second before continuing

    def solve_with_astar(self):
        start_time = time.time()
        self.solution_path = a_star(self.maze)
        elapsed_time = time.time() - start_time
        self.show_solution(elapsed_time, "A*")
        time.sleep(1)  # Pause for 1 second before continuing

    def solve_with_iddfs(self):
        start_time = time.time()
        self.solution_path = iddfs(self.maze)
        elapsed_time = time.time() - start_time
        self.show_solution(elapsed_time, "IDDFS")
        time.sleep(1)  # Pause for 1 second before continuing

    def show_solution(self, elapsed_time, algorithm_name):
        if self.solution_path:
            print(f"{algorithm_name} solved the maze in {elapsed_time:.4f} seconds and {len(self.solution_path)} steps.")
            self.draw_maze()
        else:
            print(f"{algorithm_name} could not find a solution.")


def dfs(maze):
    start = (0, 0)
    end = (len(maze) - 1, len(maze) - 1)
    stack = [start]
    visited = set()
    path = []

    while stack:
        current = stack.pop()
        path.append(current)

        if current == end:
            return path

        if current not in visited:
            visited.add(current)
            for neighbor in get_neighbors(current, maze):
                if neighbor not in visited:
                    stack.append(neighbor)
    return None


def bfs(maze):
    start = (0, 0)
    end = (len(maze) - 1, len(maze) - 1)
    queue = Queue()
    queue.put([start])
    visited = set([start])

    while not queue.empty():
        path = queue.get()
        current = path[-1]

        if current == end:
            return path

        for neighbor in get_neighbors(current, maze):
            if neighbor not in visited:
                queue.put(path + [neighbor])
                visited.add(neighbor)
    return None


def a_star(maze):
    start = (0, 0)
    end = (len(maze) - 1, len(maze) - 1)
    pq = PriorityQueue()
    pq.put((0, [start]))
    visited = set()

    while not pq.empty():
        _, path = pq.get()
        current = path[-1]

        if current == end:
            return path

        if current not in visited:
            visited.add(current)
            for neighbor in get_neighbors(current, maze):
                if neighbor not in visited:
                    new_path = path + [neighbor]
                    cost = len(new_path) + heuristic(neighbor, end)
                    pq.put((cost, new_path))
    return None


def iddfs(maze, max_depth=50):
    """Performs Iterative Deepening Depth-First Search."""
    def dls(node, depth, path):
        """Depth-Limited Search (DLS)"""
        if node == (len(maze) - 1, len(maze) - 1):  # Goal reached
            return path + [node]
        
        if depth == 0:  # Depth limit reached, return None
            return None

        x, y = node
        for neighbor in get_neighbors(node, maze):
            if neighbor not in path:  # Avoid cycles
                result = dls(neighbor, depth - 1, path + [node])
                if result:
                    return result
        return None

    # Iteratively increase depth limit
    for depth in range(1, max_depth + 1):
        result = dls((0, 0), depth, [])
        if result:  # Path found
            return result
    return None  # No solution found within depth limit


def heuristic(pos, end):
    return abs(pos[0] - end[0]) + abs(pos[1] - end[1])


def get_neighbors(pos, maze):
    row, col = pos
    neighbors = []

    if row > 0 and maze[row - 1][col] == 0:
        neighbors.append((row - 1, col))
    if row < len(maze) - 1 and maze[row + 1][col] == 0:
        neighbors.append((row + 1, col))
    if col > 0 and maze[row][col - 1] == 0:
        neighbors.append((row, col - 1))
    if col < len(maze[0]) - 1 and maze[row][col + 1] == 0:
        neighbors.append((row, col + 1))

    return neighbors


if __name__ == "__main__":
    root = tk.Tk()
    app = MazeSolverApp(root)
    root.mainloop()
